# Model Testing Pipeline

Identifying the best model (considered with best parameters of each respectively)
1.   Re-train models using both training and validation data
2.   Evaluate models against the testing data

In [1]:
!pip install ipython-autotime
%load_ext autotime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 20.0 MB/s eta 0:00:00
time: 529 µs (started: 2023-01-16 11:36:14 +00:00)


In [2]:
# Basic Libraries

import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
from datetime import datetime

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
time: 36.5 s (started: 2023-01-16 11:36:14 +00:00)


In [3]:
# Data Source

df = pd.read_csv("/content/drive/MyDrive/cleaned_gee_data.csv")
df = df.drop(columns = ['Unnamed: 0', 'BRIGHTNESS'], axis=1) # BRIGHTNESS deprecated
df.head()

,LATITUDE,LONGITUDE,ACQ_DATE,ACQ_TIME,OPEN_TIME,CLOSE_TIME,FIRE_OCCURRED,CO_MOL/M2,SO2_MOL/M2,NO2_MOL/M2,O3_MOL/M2,LOCATION,INSTRUMENT,DRY_SEASON
0,-5.466232,-0.176027,-1.866392,0.634294,0.506405,0.526945,0,-0.024223,-0.474440,-1.152277,-0.511001,-1.159086,0,1
1,-5.466232,-0.176027,-1.866392,0.634294,0.506405,0.526945,0,0.113599,-0.474440,-1.152277,-0.511001,-1.159086,0,1
2,-5.466232,-0.176027,-1.866392,0.634294,0.506405,0.526945,0,-0.024223,-0.474440,-1.361255,-0.511001,-1.159086,0,1
3,-5.466232,-0.176027,-1.866392,0.634294,0.506405,0.526945,0,0.113599,-0.474440,-1.361255,-0.511001,-1.159086,0,1
4,-5.433352,-0.197441,-1.723773,0.634294,2.286080,1.793843,0,-0.967684,0.339667,-1.251770,0.426114,-1.159086,0,1


time: 1.31 s (started: 2023-01-16 11:36:51 +00:00)


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 171893 entries, 0 to 171892
Data columns (total 14 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   LATITUDE       171893 non-null  float64
 1   LONGITUDE      171893 non-null  float64
 2   ACQ_DATE       171893 non-null  float64
 3   ACQ_TIME       171893 non-null  float64
 4   OPEN_TIME      171893 non-null  float64
 5   CLOSE_TIME     171893 non-null  float64
 6   FIRE_OCCURRED  171893 non-null  int64  
 7   CO_MOL/M2      171893 non-null  float64
 8   SO2_MOL/M2     171893 non-null  float64
 9   NO2_MOL/M2     171893 non-null  float64
 10  O3_MOL/M2      171893 non-null  float64
 11  LOCATION       171893 non-null  float64
 12  INSTRUMENT     171893 non-null  int64  
 13  DRY_SEASON     171893 non-null  int64  
dtypes: float64(11), int64(3)
memory usage: 18.4 MB
time: 36.2 ms (started: 2023-01-16 11:36:52 +00:00)


In [5]:
display(df['FIRE_OCCURRED'].value_counts())

0    170544
1      1349
Name: FIRE_OCCURRED, dtype: int64

time: 13.5 ms (started: 2023-01-16 11:36:52 +00:00)


In [6]:
X = df.drop('FIRE_OCCURRED', axis=1)
y = df['FIRE_OCCURRED']

time: 13.5 ms (started: 2023-01-16 11:36:52 +00:00)


In [7]:
# Training, Testing Split

from sklearn.model_selection import train_test_split

# 90:10

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=10, shuffle=True)

Original = [X_train, X_test, y_train, y_test] # For reference

time: 328 ms (started: 2023-01-16 11:36:52 +00:00)


In [8]:
if len(X_train)==len(y_train) and len(X_test) == len(y_test):
  print("X and y data length matching")
else:
  print("Error in data preparation pipeline")
print()
print("No. of training data = %d" % len(X_train))
print("No. of testing data = %d" % len(X_test))

X and y data length matching

No. of training data = 154703
No. of testing data = 17190
time: 6.11 ms (started: 2023-01-16 11:36:53 +00:00)


In [9]:
display(y_test.value_counts())

0    17059
1      131
Name: FIRE_OCCURRED, dtype: int64

time: 7.29 ms (started: 2023-01-16 11:36:53 +00:00)


In [10]:
# SMOTE

from collections import Counter
from imblearn.over_sampling import SMOTE 

print('Original dataset shape %s' % Counter(y_train))
sm = SMOTE(random_state=10)
X_train, y_train = sm.fit_resample(X_train, y_train)
print('Resampled dataset shape %s' % Counter(y_train))

Original dataset shape Counter({0: 153485, 1: 1218})
Resampled dataset shape Counter({0: 153485, 1: 153485})
time: 847 ms (started: 2023-01-16 11:36:53 +00:00)


In [11]:
# Shuffle Data since SMOTE appended many 1s at the end
# Required for some algorithms such as ANN

from sklearn.utils import shuffle

X_train, y_train = shuffle(X_train, y_train, random_state = 10)

time: 83.4 ms (started: 2023-01-16 11:36:54 +00:00)


In [12]:
# Evaluation Metrics

from sklearn.metrics import confusion_matrix, recall_score, f1_score, roc_auc_score, accuracy_score

def evaluation_metrics(y_true, y_pred):
  cfm = confusion_matrix(y_true, y_pred).ravel()
  acc = accuracy_score(y_true, y_pred)
  recs = recall_score(y_true, y_pred, average='binary')
  f1s = f1_score(y_true, y_pred, average='binary')
  rocs = roc_auc_score(y_true, y_pred, average='macro')
  return [cfm, acc, recs, f1s, rocs]

time: 1.8 ms (started: 2023-01-16 11:36:54 +00:00)


Confusion matrix format : [ tn , fp , fn , tp ]

In [13]:
# Store Model Parameters and Eval

models_final = pd.DataFrame(columns = ['model_name', 'model', 'parameters'])
models_test = pd.DataFrame(columns = ['model_name', 'confusion_matrix', 'accuracy', 'recall', 'f1_score', 'roc_auc_score'])

time: 6.47 ms (started: 2023-01-16 11:36:54 +00:00)


In [14]:
# Import ML Algorithms

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
import xgboost
from xgboost import XGBClassifier
import lightgbm
from lightgbm import LGBMClassifier
import tensorflow as tf
from tensorflow import keras
from sklearn.ensemble import VotingClassifier

# Save Model

import pickle

time: 4 s (started: 2023-01-16 11:36:54 +00:00)


## Logistic Regression

- Library: Scikit-learn

**Best Parameters:**

{'warm_start': True,
 'solver': 'lbfgs',
 'penalty': 'none',
 'max_iter': 331,
 'dual': False,
 'C': 0}

In [15]:
name = 'log_clf'

log_clf = LogisticRegression(penalty = 'none', 
                             warm_start = True,
                             solver = 'lbfgs',
                             max_iter = 331,
                             dual = False,
                             C = 0,
                             n_jobs = -1, 
                             random_state = 10
                             ).fit(X_train,y_train)

y_true = y_test
y_pred = log_clf.predict(X_test)
evaluation_results = evaluation_metrics(y_true, y_pred)

models_final = models_final.append({'model_name': name, 
                        'model': log_clf, 
                        'parameters': log_clf.get_params()}, 
                        ignore_index=True)

models_test = models_test.append({'model_name': name, 
                                  'confusion_matrix' : evaluation_results[0], 
                                  'accuracy': evaluation_results[1], 
                                  'recall' : evaluation_results[2], 
                                  'f1_score': evaluation_results[3],
                                  'roc_auc_score': evaluation_results[4]}, 
                                 ignore_index=True)

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(


time: 5.78 s (started: 2023-01-16 11:36:58 +00:00)


In [16]:
# Save Model
pickle.dump(log_clf, open('log_clf.sav', 'wb')) 

# Load Model
# log_clf = pickle.load(open('log_clf.sav', 'rb'))

time: 1.39 ms (started: 2023-01-16 11:37:04 +00:00)


## Support Vector Machine (SVM)

- Library: Scikit-learn

**Best Parameters:**

{
'kernel': 'rbf',
'C' : '8',
'class_weight'='balanced'
}

In [17]:
X_train_SVM = Original[0]
X_test_SVM = Original[1]
y_train_SVM = Original[2]
y_test_SVM = Original[3]

time: 1.42 ms (started: 2023-01-16 11:37:04 +00:00)


In [18]:
# Shuffle

X_train_SVM, y_train_SVM = shuffle(X_train_SVM, y_train_SVM, random_state = 10)

time: 60.2 ms (started: 2023-01-16 11:37:04 +00:00)


In [19]:
display(y_train_SVM.value_counts())

0    153485
1      1218
Name: FIRE_OCCURRED, dtype: int64

time: 11.7 ms (started: 2023-01-16 11:37:04 +00:00)


In [20]:
name = 'svc_clf'

svc_clf = SVC(kernel = 'rbf', 
              C = 8,
              class_weight='balanced',
              random_state = 10
              ).fit(X_train_SVM,y_train_SVM)

y_true = y_test_SVM
y_pred = svc_clf.predict(X_test_SVM)
evaluation_results = evaluation_metrics(y_true, y_pred)

models_final = models_final.append({'model_name': name, 
                        'model': svc_clf, 
                        'parameters': svc_clf.get_params()}, 
                       ignore_index=True)

models_test = models_test.append({'model_name': name, 
                                  'confusion_matrix' : evaluation_results[0], 
                                  'accuracy': evaluation_results[1], 
                                  'recall' : evaluation_results[2], 
                                  'f1_score': evaluation_results[3],
                                  'roc_auc_score': evaluation_results[4]}, 
                                 ignore_index=True)

time: 5min 47s (started: 2023-01-16 11:37:04 +00:00)


In [21]:
# Save Model
pickle.dump(svc_clf, open('svc_clf.sav', 'wb')) 

# Load Model
# svc_clf = pickle.load(open('svc_clf.sav', 'rb'))

time: 6.23 ms (started: 2023-01-16 11:42:51 +00:00)


## Naive Bayes

- Library: Scikit-learn

**Best Parameters:**

{'var_smoothing': 1e-3}

In [22]:
name = 'bayes_clf'

bayes_clf = GaussianNB(var_smoothing = 1e-3
                       ).fit(X_train,y_train)

y_true = y_test
y_pred = bayes_clf.predict(X_test)
evaluation_results = evaluation_metrics(y_true, y_pred)

models_final = models_final.append({'model_name': name, 
                        'model': bayes_clf, 
                        'parameters': bayes_clf.get_params()}, 
                       ignore_index=True)

models_test = models_test.append({'model_name': name, 
                                  'confusion_matrix' : evaluation_results[0], 
                                  'accuracy': evaluation_results[1], 
                                  'recall' : evaluation_results[2], 
                                  'f1_score': evaluation_results[3],
                                  'roc_auc_score': evaluation_results[4]}, 
                                 ignore_index=True)

time: 166 ms (started: 2023-01-16 11:42:51 +00:00)


In [23]:
# Save Model
pickle.dump(bayes_clf, open('bayes_clf.sav', 'wb')) 

# Load Model
# bayes_clf = pickle.load(open('bayes_clf.sav', 'rb'))

time: 1.57 ms (started: 2023-01-16 11:42:52 +00:00)


## K-Nearest Neighbor

- Library: Scikit-learn

**Best Parameters:**

{'n_neighbors': 19, 'algorithm': 'kd_tree'}

In [24]:
name = 'neigh_clf'

neigh_clf = KNeighborsClassifier(n_neighbors = 19, 
                                 algorithm = 'kd_tree',
                                 n_jobs = -1, 
                                 ).fit(X_train,y_train)

y_true = y_test
y_pred = neigh_clf.predict(X_test)
evaluation_results = evaluation_metrics(y_true, y_pred)

models_final = models_final.append({'model_name': name, 
                        'model': neigh_clf, 
                        'parameters': neigh_clf.get_params()}, 
                       ignore_index=True)

models_test = models_test.append({'model_name': name, 
                                  'confusion_matrix' : evaluation_results[0], 
                                  'accuracy': evaluation_results[1], 
                                  'recall' : evaluation_results[2], 
                                  'f1_score': evaluation_results[3],
                                  'roc_auc_score': evaluation_results[4]}, 
                                 ignore_index=True)

time: 14.4 s (started: 2023-01-16 11:42:52 +00:00)


In [88]:
# Save Model
# pickle.dump(neigh_clf, open('neigh_clf.sav', 'wb')) 
pickle.dump(neigh_clf, open('neigh_clf.h', 'wb')) 

# Load Model
# neigh_clf = pickle.load(open('neigh_clf.sav', 'rb'))

time: 408 ms (started: 2023-01-16 14:29:46 +00:00)


## Decision Tree

- Library: Scikit-learn

**Best Parameters:**

{'splitter': 'random',
 'min_samples_leaf': 2,
 'max_features': 11,
 'max_depth': None,
 'criterion': 'entropy'}

In [26]:
name = 'tree_clf'

tree_clf = DecisionTreeClassifier(criterion = 'entropy', 
                                  splitter = 'random', 
                                  min_samples_leaf = 2,
                                  max_features = 11,
                                  max_depth = None,
                                  random_state = 10
                                  ).fit(X_train,y_train)

y_true = y_test
y_pred = tree_clf.predict(X_test)
evaluation_results = evaluation_metrics(y_true, y_pred)

models_final = models_final.append({'model_name': name, 
                        'model': tree_clf, 
                        'parameters': tree_clf.get_params()}, 
                       ignore_index=True)

models_test = models_test.append({'model_name': name, 
                                  'confusion_matrix' : evaluation_results[0], 
                                  'accuracy': evaluation_results[1], 
                                  'recall' : evaluation_results[2], 
                                  'f1_score': evaluation_results[3],
                                  'roc_auc_score': evaluation_results[4]}, 
                                 ignore_index=True)

time: 698 ms (started: 2023-01-16 11:43:06 +00:00)


In [27]:
# Save Model
pickle.dump(tree_clf, open('tree_clf.sav', 'wb')) 

# Load Model
# tree_clf = pickle.load(open('tree_clf.sav', 'rb'))

time: 3.8 ms (started: 2023-01-16 11:43:07 +00:00)


## Random Forest Classifier

- Library: Scikit-learn

**Best Parameters:**

{'n_estimators': 415,
 'min_samples_split': 6,
 'min_samples_leaf': 1,
 'max_features': 4,
 'max_depth': 18}

In [28]:
name = 'rnd_clf'

rnd_clf = RandomForestClassifier(n_estimators = 415, 
                                  min_samples_split = 6,
                                  min_samples_leaf = 1,
                                  max_features = 4,
                                  max_depth = 18, 
                                  n_jobs = -1, 
                                  random_state = 10
                                  ).fit(X_train,y_train)

y_true = y_test
y_pred = rnd_clf.predict(X_test)
evaluation_results = evaluation_metrics(y_true, y_pred)

models_final = models_final.append({'model_name': name, 
                        'model': rnd_clf, 
                        'parameters': rnd_clf.get_params()}, 
                       ignore_index=True)

models_test = models_test.append({'model_name': name, 
                                  'confusion_matrix' : evaluation_results[0], 
                                  'accuracy': evaluation_results[1], 
                                  'recall' : evaluation_results[2], 
                                  'f1_score': evaluation_results[3],
                                  'roc_auc_score': evaluation_results[4]}, 
                                 ignore_index=True)

time: 5min 12s (started: 2023-01-16 11:43:07 +00:00)


In [80]:
# Save Model
pickle.dump(rnd_clf, open('rnd_clf.sav', 'wb')) 


# Load Model
# rnd_clf = pickle.load(open('rnd_clf.sav', 'rb'))

time: 389 ms (started: 2023-01-16 14:23:07 +00:00)


## Gradient Boosting Classifier

- Library: Scikit-learn

**Best Parameters:**

{'n_estimators': 1000, 'max_depth': 8, 'learning_rate': 0.1}

In [30]:
name = 'gboost_clf'

gboost_clf = GradientBoostingClassifier(n_estimators = 1000, 
                                        learning_rate = 0.1, 
                                        max_depth = 8,
                                        random_state = 10
                                        ).fit(X_train,y_train)

y_true = y_test
y_pred = gboost_clf.predict(X_test)
evaluation_results = evaluation_metrics(y_true, y_pred)

models_final = models_final.append({'model_name': name, 
                        'model': gboost_clf, 
                        'parameters': gboost_clf.get_params()}, 
                       ignore_index=True)

models_test = models_test.append({'model_name': name, 
                                  'confusion_matrix' : evaluation_results[0], 
                                  'accuracy': evaluation_results[1], 
                                  'recall' : evaluation_results[2], 
                                  'f1_score': evaluation_results[3],
                                  'roc_auc_score': evaluation_results[4]}, 
                                 ignore_index=True)

time: 47min (started: 2023-01-16 11:48:20 +00:00)


In [84]:
# Save Model
# pickle.dump(gboost_clf, open('gboost_clf.sav', 'wb')) 
pickle.dump(gboost_clf, open('gboost_clf.h', 'wb')) 

# Load Model
# gboost_clf = pickle.load(open('gboost_clf.sav', 'rb'))

time: 186 ms (started: 2023-01-16 14:24:25 +00:00)


## XGBoost

- Library: xgboost

**Best Parameters:**

{'n_estimators': 1000,
 'min_child_weight': 7,
 'max_depth': 8,
 'learning_rate': 0.1}

In [32]:
name = 'xgboost_clf'

xgboost_clf = XGBClassifier(n_estimators = 1000, 
                            learning_rate = 0.1,
                            max_depth = 8, 
                            min_child_weight = 7,
                            random_state = 10
                            ).fit(X_train,y_train)

y_true = y_test
y_pred = xgboost_clf.predict(X_test)
evaluation_results = evaluation_metrics(y_true, y_pred)

models_final = models_final.append({'model_name': name, 
                        'model': xgboost_clf, 
                        'parameters': xgboost_clf.get_params()}, 
                       ignore_index=True)

models_test = models_test.append({'model_name': name, 
                                  'confusion_matrix' : evaluation_results[0], 
                                  'accuracy': evaluation_results[1], 
                                  'recall' : evaluation_results[2], 
                                  'f1_score': evaluation_results[3],
                                  'roc_auc_score': evaluation_results[4]}, 
                                 ignore_index=True)

time: 11min 14s (started: 2023-01-16 12:35:21 +00:00)


In [82]:
# Save Model
# pickle.dump(xgboost_clf, open('xgboost_clf.sav', 'wb')) 
pickle.dump(xgboost_clf, open('xgboost_clf.h', 'wb')) 

# Load Model
# xgboost_clf = pickle.load(open('xgboost_clf.sav', 'rb'))

time: 16.2 ms (started: 2023-01-16 14:24:03 +00:00)


## LightGBM

- Library: lightbgm

**Best Parameters:**

{'num_leaves': 50,
 'n_estimators': 1000,
 'min_data_in_leaf': 10,
 'max_depth': 8,
 'learning_rate': 0.05}

In [34]:
name = 'lightgbm_clf'

lightgbm_clf = LGBMClassifier(n_estimators = 1000, 
                              learning_rate = 0.05, 
                              max_depth = 8,
                              num_leaves = 50,
                              min_data_in_leaf = 10,
                              random_state = 10
                              ).fit(X_train,y_train)

y_true = y_test
y_pred = lightgbm_clf.predict(X_test)
evaluation_results = evaluation_metrics(y_true, y_pred)

models_final = models_final.append({'model_name': name, 
                        'model': lightgbm_clf, 
                        'parameters': lightgbm_clf.get_params()}, 
                       ignore_index=True)

models_test = models_test.append({'model_name': name, 
                                  'confusion_matrix' : evaluation_results[0], 
                                  'accuracy': evaluation_results[1], 
                                  'recall' : evaluation_results[2], 
                                  'f1_score': evaluation_results[3],
                                  'roc_auc_score': evaluation_results[4]}, 
                                 ignore_index=True)

time: 46.7 s (started: 2023-01-16 12:46:36 +00:00)


In [35]:
# Save Model
pickle.dump(lightgbm_clf, open('lightgbm_clf.sav', 'wb')) 

# Load Model
# lightgbm_clf = pickle.load(open('lightgbm_clf.sav', 'rb'))

time: 274 ms (started: 2023-01-16 12:47:22 +00:00)


## Artificial Neural Network

- Library: Keras, Tensorflow

**Best Parameters:**
- Batch size 15
- EPOCH 50

In [36]:
tf.random.set_seed(10)

time: 1.47 ms (started: 2023-01-16 12:47:23 +00:00)


In [37]:
name = 'ann_clf'

ann_clf = keras.models.Sequential([
    keras.layers.Dense(15, input_shape=(X_train.shape[1],), activation='relu'), # No bias term
    # keras.layers.Dense(10, activation='relu'), 
    keras.layers.Dense(10, activation='relu'), 
    keras.layers.Dense(1, activation='sigmoid')
])

ann_clf.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 15)                210       
                                                                 
 dense_1 (Dense)             (None, 10)                160       
                                                                 
 dense_2 (Dense)             (None, 1)                 11        
                                                                 
Total params: 381
Trainable params: 381
Non-trainable params: 0
_________________________________________________________________
time: 214 ms (started: 2023-01-16 12:47:23 +00:00)


In [38]:
display(y_train.value_counts())

1    153485
0    153485
Name: FIRE_OCCURRED, dtype: int64

time: 21.6 ms (started: 2023-01-16 12:47:23 +00:00)


In [39]:
display(y_test.value_counts())

0    17059
1      131
Name: FIRE_OCCURRED, dtype: int64

time: 11.2 ms (started: 2023-01-16 12:47:23 +00:00)


In [40]:
ann_clf.compile(optimizer = 'adam', 
                metrics=['accuracy'], 
                loss ='binary_crossentropy')

record = ann_clf.fit(
            X_train, 
            y_train, 
            validation_data = (X_test, y_test), 
            batch_size = 15, 
            epochs = 50)

Epoch 1/50
20465/20465 [==============================] - 51s 2ms/step - loss: 0.2060 - accuracy: 0.9237 - val_loss: 0.2158 - val_accuracy: 0.9000
Epoch 2/50
20465/20465 [==============================] - 50s 2ms/step - loss: 0.1613 - accuracy: 0.9418 - val_loss: 0.1874 - val_accuracy: 0.9163
Epoch 3/50
20465/20465 [==============================] - 50s 2ms/step - loss: 0.1538 - accuracy: 0.9437 - val_loss: 0.1529 - val_accuracy: 0.9328
Epoch 4/50
20465/20465 [==============================] - 48s 2ms/step - loss: 0.1492 - accuracy: 0.9463 - val_loss: 0.1872 - val_accuracy: 0.9208
Epoch 5/50
20465/20465 [==============================] - 50s 2ms/step - loss: 0.1453 - accuracy: 0.9485 - val_loss: 0.1644 - val_accuracy: 0.9313
Epoch 6/50
20465/20465 [==============================] - 48s 2ms/step - loss: 0.1416 - accuracy: 0.9501 - val_loss: 0.1646 - val_accuracy: 0.9300
Epoch 7/50
20465/20465 [==============================] - 46s 2ms/step - loss: 0.1385 - accuracy: 0.9515 - val_loss: 0

In [41]:
y_true = y_test
prediction = ann_clf.predict(X_test)
prediction = pd.Series(prediction[:, 0])
y_pred = []

for i in range(len(prediction)):
  if prediction[i] >= 0.5:
    y_pred.append(1)
  else:
    y_pred.append(0)

evaluation_results = evaluation_metrics(y_true, y_pred)

models_final = models_final.append({'model_name': name, 
                        'model': ann_clf, 
                        'parameters': ann_clf.layers}, 
                       ignore_index=True)

models_test = models_test.append({'model_name': name, 
                                  'confusion_matrix' : evaluation_results[0], 
                                  'accuracy': evaluation_results[1], 
                                  'recall' : evaluation_results[2], 
                                  'f1_score': evaluation_results[3],
                                  'roc_auc_score': evaluation_results[4]}, 
                                 ignore_index=True)

538/538 [==============================] - 1s 1ms/step
time: 1.18 s (started: 2023-01-16 13:30:46 +00:00)


In [42]:
# Save Model
ann_clf.save('ann_clf.h5') 

# Load Model
# ann_clf = tf.keras.models.load_model('ann_clf.h5')

time: 28.9 ms (started: 2023-01-16 13:30:47 +00:00)


## Voting Classifier
- Library: Scikit-learn, Keras, Tensorflow

**Best Parameters:**

{'voting': 'hard'}

In [43]:
display(models_test)

,model_name,confusion_matrix,accuracy,recall,f1_score,roc_auc_score
0,log_clf,"[13920, 3139, 39, 92]",0.815125,0.702290,0.054729,0.759141
1,svc_clf,"[16301, 758, 14, 117]",0.955090,0.893130,0.232604,0.924348
2,bayes_clf,"[11200, 5859, 14, 117]",0.658348,0.893130,0.038317,0.774837
3,neigh_clf,"[16553, 506, 12, 119]",0.969866,0.908397,0.314815,0.939368
4,tree_clf,"[16927, 132, 22, 109]",0.991041,0.832061,0.586022,0.912162
5,rnd_clf,"[16823, 236, 19, 112]",0.985166,0.854962,0.467641,0.920564
6,gboost_clf,"[17008, 51, 21, 110]",0.995812,0.839695,0.753425,0.918353
7,xgboost_clf,"[17001, 58, 22, 109]",0.995346,0.832061,0.731544,0.914331
8,lightgbm_clf,"[16976, 83, 22, 109]",0.993892,0.832061,0.674923,0.913598
9,ann_clf,"[16078, 981, 14, 117]",0.942118,0.893130,0.190399,0.917812


time: 22.9 ms (started: 2023-01-16 13:30:47 +00:00)


In [44]:
print('Best Model By Accuracy')
print(models_test.loc[models_test['accuracy'] == max(models_test['accuracy'])].model_name.to_string(index=False))
print('-----------------------')
print('Best Model By Recall')
print(models_test.loc[models_test['recall'] == max(models_test['recall'])].model_name.to_string(index=False))
print('-----------------------')
print('Best Model By F1')
print(models_test.loc[models_test['f1_score'] == max(models_test['f1_score'])].model_name.to_string(index=False))
print('-----------------------')
print('Best Model By ROC')
print(models_test.loc[models_test['roc_auc_score'] == max(models_test['roc_auc_score'])].model_name.to_string(index=False))
print('-----------------------')

Best Model By Accuracy
gboost_clf
-----------------------
Best Model By Recall
neigh_clf
-----------------------
Best Model By F1
gboost_clf
-----------------------
Best Model By ROC
neigh_clf
-----------------------
time: 17.2 ms (started: 2023-01-16 13:30:47 +00:00)


In [ ]:
name = 'ensem_clf'

ensem_clf = VotingClassifier(estimators=[('m1', neigh_clf), ('m2', gboost_clf), ('m3', xgboost_clf)],
                             voting = 'hard',
                             n_jobs = -1, 
                             ).fit(X_train,y_train)

y_true = y_test
y_pred = ensem_clf.predict(X_test)
evaluation_results = evaluation_metrics(y_true, y_pred)

models_final = models_final.append({'model_name': name, 
                        'model': ensem_clf, 
                        'parameters': ensem_clf.get_params()}, 
                       ignore_index=True)

models_test = models_test.append({'model_name': name, 
                                  'confusion_matrix' : evaluation_results[0], 
                                  'accuracy': evaluation_results[1], 
                                  'recall' : evaluation_results[2], 
                                  'f1_score': evaluation_results[3],
                                  'roc_auc_score': evaluation_results[4]}, 
                                 ignore_index=True)

In [ ]:
# Save Model
pickle.dump(ensem_clf, open('ensem_clf.sav', 'wb')) 

# Load Model
# ensem_clf = pickle.load(open('ensem_clf.sav', 'rb'))

# Model Testing Result

### Results

In [46]:
display(models_final)

,model_name,model,parameters
0,log_clf,"LogisticRegression(C=0, max_iter=331, n_jobs=-...","{'C': 0, 'class_weight': None, 'dual': False, ..."
1,svc_clf,"SVC(C=8, class_weight='balanced', random_state...","{'C': 8, 'break_ties': False, 'cache_size': 20..."
2,bayes_clf,GaussianNB(var_smoothing=0.001),"{'priors': None, 'var_smoothing': 0.001}"
3,neigh_clf,"KNeighborsClassifier(algorithm='kd_tree', n_jo...","{'algorithm': 'kd_tree', 'leaf_size': 30, 'met..."
4,tree_clf,"DecisionTreeClassifier(criterion='entropy', ma...","{'ccp_alpha': 0.0, 'class_weight': None, 'crit..."
5,rnd_clf,"(DecisionTreeClassifier(max_depth=18, max_feat...","{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w..."
6,gboost_clf,([DecisionTreeRegressor(criterion='friedman_ms...,"{'ccp_alpha': 0.0, 'criterion': 'friedman_mse'..."
7,xgboost_clf,"XGBClassifier(max_depth=8, min_child_weight=7,...","{'base_score': 0.5, 'booster': 'gbtree', 'cols..."
8,lightgbm_clf,"LGBMClassifier(learning_rate=0.05, max_depth=8...","{'boosting_type': 'gbdt', 'class_weight': None..."
9,ann_clf,<keras.engine.sequential.Sequential object at ...,[<keras.layers.core.dense.Dense object at 0x7f...


time: 454 ms (started: 2023-01-16 13:45:27 +00:00)


In [47]:
display(models_test)

,model_name,confusion_matrix,accuracy,recall,f1_score,roc_auc_score
0,log_clf,"[13920, 3139, 39, 92]",0.815125,0.702290,0.054729,0.759141
1,svc_clf,"[16301, 758, 14, 117]",0.955090,0.893130,0.232604,0.924348
2,bayes_clf,"[11200, 5859, 14, 117]",0.658348,0.893130,0.038317,0.774837
3,neigh_clf,"[16553, 506, 12, 119]",0.969866,0.908397,0.314815,0.939368
4,tree_clf,"[16927, 132, 22, 109]",0.991041,0.832061,0.586022,0.912162
5,rnd_clf,"[16823, 236, 19, 112]",0.985166,0.854962,0.467641,0.920564
6,gboost_clf,"[17008, 51, 21, 110]",0.995812,0.839695,0.753425,0.918353
7,xgboost_clf,"[17001, 58, 22, 109]",0.995346,0.832061,0.731544,0.914331
8,lightgbm_clf,"[16976, 83, 22, 109]",0.993892,0.832061,0.674923,0.913598
9,ann_clf,"[16078, 981, 14, 117]",0.942118,0.893130,0.190399,0.917812


time: 38.3 ms (started: 2023-01-16 13:45:28 +00:00)


In [48]:
print('Best Model By Accuracy')
print(models_test.loc[models_test['accuracy'] == max(models_test['accuracy'])].model_name.to_string(index=False))
print('-----------------------')
print('Best Model By Recall')
print(models_test.loc[models_test['recall'] == max(models_test['recall'])].model_name.to_string(index=False))
print('-----------------------')
print('Best Model By F1')
print(models_test.loc[models_test['f1_score'] == max(models_test['f1_score'])].model_name.to_string(index=False))
print('-----------------------')
print('Best Model By ROC')
print(models_test.loc[models_test['roc_auc_score'] == max(models_test['roc_auc_score'])].model_name.to_string(index=False))
print('-----------------------')

Best Model By Accuracy
gboost_clf
-----------------------
Best Model By Recall
neigh_clf
-----------------------
Best Model By F1
gboost_clf
-----------------------
Best Model By ROC
neigh_clf
-----------------------
time: 39.9 ms (started: 2023-01-16 13:45:28 +00:00)


In [50]:
models_final.to_csv('models_testing.csv')
!cp models_testing.csv "drive/My Drive/"

time: 263 ms (started: 2023-01-16 13:45:41 +00:00)


In [51]:
models_test.to_csv('models_testing_eval.csv')
!cp models_testing_eval.csv "drive/My Drive/"

time: 225 ms (started: 2023-01-16 13:45:42 +00:00)


In [63]:
result = models_test.reindex([0,1,2,3,4,5,7,6,8,9])
result.loc[6,'model_name'] = 'xgboost_clf'
result.loc[7,'model_name'] = 'gboost_clf'
display(result)

,model_name,confusion_matrix,accuracy,recall,f1_score,roc_auc_score
0,log_clf,"[13920, 3139, 39, 92]",0.815125,0.702290,0.054729,0.759141
1,svc_clf,"[16301, 758, 14, 117]",0.955090,0.893130,0.232604,0.924348
2,bayes_clf,"[11200, 5859, 14, 117]",0.658348,0.893130,0.038317,0.774837
3,neigh_clf,"[16553, 506, 12, 119]",0.969866,0.908397,0.314815,0.939368
4,tree_clf,"[16927, 132, 22, 109]",0.991041,0.832061,0.586022,0.912162
5,rnd_clf,"[16823, 236, 19, 112]",0.985166,0.854962,0.467641,0.920564
7,gboost_clf,"[17001, 58, 22, 109]",0.995346,0.832061,0.731544,0.914331
6,xgboost_clf,"[17008, 51, 21, 110]",0.995812,0.839695,0.753425,0.918353
8,lightgbm_clf,"[16976, 83, 22, 109]",0.993892,0.832061,0.674923,0.913598
9,ann_clf,"[16078, 981, 14, 117]",0.942118,0.893130,0.190399,0.917812


time: 62.3 ms (started: 2023-01-16 14:05:09 +00:00)


In [64]:
print('Best Model By Accuracy')
print(result.loc[result['accuracy'] == max(result['accuracy'])].model_name.to_string(index=False))
print('-----------------------')
print('Best Model By Recall')
print(result.loc[result['recall'] == max(result['recall'])].model_name.to_string(index=False))
print('-----------------------')
print('Best Model By F1')
print(result.loc[result['f1_score'] == max(result['f1_score'])].model_name.to_string(index=False))
print('-----------------------')
print('Best Model By ROC')
print(result.loc[result['roc_auc_score'] == max(result['roc_auc_score'])].model_name.to_string(index=False))
print('-----------------------')

Best Model By Accuracy
xgboost_clf
-----------------------
Best Model By Recall
neigh_clf
-----------------------
Best Model By F1
xgboost_clf
-----------------------
Best Model By ROC
neigh_clf
-----------------------
time: 28.4 ms (started: 2023-01-16 14:05:43 +00:00)


In [66]:
result.to_csv('model_testing_result.csv')
!cp model_testing_result.csv "drive/My Drive/"

time: 180 ms (started: 2023-01-16 14:07:42 +00:00)


xgboost_clf is the most suitable model

### Model Export

In [ ]:
from google.colab import files

In [ ]:
files.download('log_clf.sav')

In [ ]:
files.download('svc_clf.sav')

In [ ]:
files.download('bayes_clf.sav')

In [ ]:
files.download('neigh_clf.sav')

In [ ]:
files.download('tree_clf.sav')

In [ ]:
files.download('rnd_clf.sav')

In [ ]:
files.download('gboost_clf.sav')

In [ ]:
files.download('xgboost_clf.sav')

In [ ]:
files.download('lightgbm_clf.sav')

In [ ]:
files.download('ann_clf.h5')